<a href="https://colab.research.google.com/github/SteakEnthusiast/APS360-Team-29/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train_folder_path = "/content/drive/MyDrive/APS360_Project/train_set"
validation_folder_path = "/content/drive/MyDrive/APS360_Project/val_set"
test_folder_path = "/content/drive/MyDrive/APS360_Project/test_set"

In [35]:
# resize to 32 by 32 and convert to tensor
transformation = transforms.Compose([
            transforms.Resize((32,32)),
            transforms.ToTensor()
])

In [51]:
# load datasets and apply transformation
train_dataset = datasets.ImageFolder(root=train_folder_path, transform=transformation)
validation_dataset = datasets.ImageFolder(root=validation_folder_path, transform=transformation)
test_dataset = datasets.ImageFolder(root=test_folder_path, transform=transformation)

In [52]:
# get features by simply flattening pixel values
def extract_features(dataset):
  Features = []
  Labels = []
  for image, label in dataset:
    features = np.array(image.flatten())
    Features.append(features)
    Labels.append(label)
  Features = np.array(Features)
  Labels = np.array(Labels)
  return Features, Labels

In [53]:
train_features, train_labels = extract_features(train_dataset)
validation_features, validation_labels = extract_features(validation_dataset)
test_features, test_labels = extract_features(test_dataset)

In [54]:
# define the number of neigbhours
k = 2

# get the model
baseline_model = KNeighborsClassifier(n_neighbors=k)

# "train" the model (store the trainset in this case)
baseline_model.fit(train_features, train_labels)

# make predictions
validation_predicts = baseline_model.predict(validation_features)
test_predicts = baseline_model.predict(test_features)

In [55]:
# get accruacy
validation_accuracy = accuracy_score(validation_predicts, validation_labels)
test_accuracy = accuracy_score(test_predicts, test_labels)
print(f"validation accuracy is {validation_accuracy}")
print(f"test accuracy is {test_accuracy}")

validation accuracy is 0.6213872832369942
test accuracy is 0.6058823529411764


In [56]:
# use more neighbours
k = 4

# get the model
baseline_model = KNeighborsClassifier(n_neighbors=k)

# "train" the model (store the trainset in this case)
baseline_model.fit(train_features, train_labels)

# make predictions
validation_predicts = baseline_model.predict(validation_features)
test_predicts = baseline_model.predict(test_features)

In [57]:
# get accruacy
validation_accuracy = accuracy_score(validation_predicts, validation_labels)
test_accuracy = accuracy_score(test_predicts, test_labels)
print(f"validation accuracy is {validation_accuracy}")
print(f"test accuracy is {test_accuracy}")

validation accuracy is 0.6213872832369942
test accuracy is 0.5705882352941176
